In [1]:
import os

# 定义基础路径
base_dir = 'archive/Pest24/VOCdevkit/voc2007'
imagesets_dir = os.path.join(base_dir, 'ImageSets')
yolo_path_dir = os.path.join(base_dir, 'yolo_path')
images_dir = os.path.join(base_dir, 'images') # 假设图片在 images 目录

# 确保 yolo_path 目录存在
os.makedirs(yolo_path_dir, exist_ok=True)

# 处理 train, val, test 文件
for split in ['train', 'val', 'test']:
    imageset_file = os.path.join(imagesets_dir, f'{split}.txt')
    yolo_path_file = os.path.join(yolo_path_dir, f'{split}.txt')

    if not os.path.exists(imageset_file):
        print(f"警告：找不到文件 {imageset_file}")
        continue

    with open(imageset_file, 'r') as f:
        image_names = f.read().strip().split()

    # 写入相对路径
    with open(yolo_path_file, 'w') as f:
        for name in image_names:
            # 假设图片是 .jpg 格式
            image_path = os.path.join(images_dir, f'{name}.jpg').replace('\\', '/')
            f.write(image_path + '\n')
    
    print(f"已成功生成 {yolo_path_file}")

print("\n所有路径文件已更新完毕。")


已成功生成 archive/Pest24/VOCdevkit/voc2007\yolo_path\train.txt
已成功生成 archive/Pest24/VOCdevkit/voc2007\yolo_path\val.txt
已成功生成 archive/Pest24/VOCdevkit/voc2007\yolo_path\test.txt

所有路径文件已更新完毕。


In [2]:
import os

# 返回到项目根目录
os.chdir('d:\\.FileModel')

# 列出 yolo_path 目录的内容
yolo_path_dir = 'archive/Pest24/VOCdevkit/voc2007/yolo_path'
if os.path.exists(yolo_path_dir):
    files = os.listdir(yolo_path_dir)
    print(f"'{yolo_path_dir}' 目录中的文件：")
    for file in files:
        print(file)
else:
    print(f"目录 '{yolo_path_dir}' 不存在。")

'archive/Pest24/VOCdevkit/voc2007/yolo_path' 目录中的文件：
test.cache
test.txt
train.cache
train.txt
val.txt


# 阶段一：Warmup 训练

本阶段只训练模型的头部，冻结主干网络，使用较小学习率，防止破坏预训练特征。

- 预训练模型：yolo11n.pt
- 数据集配置：pest24.yaml
- 冻结主干网络（backbone）
- 只训练头部（head）
- 学习率较小
- 训练周期较短
- 启用权重衰减、LoRA等正则化


In [2]:
import sys
import ultralytics
print(sys.executable)
print(ultralytics.__file__)
from ultralytics import YOLO
print("Successfully imported YOLO")

D:\.FileModel\.venv\Scripts\python.exe
D:\.FileModel\.venv\Lib\site-packages\ultralytics\__init__.py
Successfully imported YOLO


In [3]:
# 检查 GPU/CUDA 可用性并打印详细信息
import sys
import torch
import subprocess
import shutil

print(f"Python executable: {sys.executable}")
print(f"torch.__version__: {torch.__version__}")

cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")
print(f"CUDA device count: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    try:
        name = torch.cuda.get_device_name(i)
    except Exception:
        name = 'Unknown'
    print(f"GPU {i}: {name}")

# 尝试调用 nvidia-smi 获取更详细的状态（如果可用）
if shutil.which('nvidia-smi'):
    try:
        out = subprocess.check_output(['nvidia-smi', '--query-gpu=index,name,memory.total,memory.free,driver_version', '--format=csv,noheader,nounits'], stderr=subprocess.STDOUT)
        print('\nnvidia-smi output:')
        print(out.decode())
    except subprocess.CalledProcessError as e:
        print('nvidia-smi command failed:', e.output.decode() if e.output else e)
    except Exception as e:
        print('nvidia-smi query error:', e)
else:
    print('nvidia-smi not found in PATH')

# 建议用于 model.train 的 device 变量
device = 0 if cuda_available and torch.cuda.device_count() > 0 else 'cpu'
print(f"Recommended training device: {device}")

Python executable: D:\.FileModel\.venv\Scripts\python.exe
torch.__version__: 2.8.0+cu126
CUDA available: True
CUDA device count: 1
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU

nvidia-smi output:
0, NVIDIA GeForce RTX 4060 Laptop GPU, 8188, 7495, 560.94

Recommended training device: 0


In [4]:
# 训练参数优化探索
import psutil
import torch
from math import ceil
import gc
import sys
import subprocess
from pathlib import Path
import datetime

# 主动清理内存
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

def estimate_batch_size():
    # 获取系统内存信息
    mem = psutil.virtual_memory()
    gpu_mem = 0
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
    
    # 估算合理的batch size
    # 假设每张图片约占用 100MB 显存/内存
    if gpu_mem > 0:
        max_batch = int(gpu_mem * 0.7 / 0.1)  # 使用70%显存
    else:
        max_batch = int(mem.available / 1024**3 * 0.5 / 0.1)  # 使用50%可用内存
    
    # 确保batch size是8的倍数（有利于GPU优化）
    batch_size = max(8, min(32, 8 * ceil(max_batch/8)))
    return batch_size

def estimate_workers():
    # 获取CPU核心数
    cpu_count = psutil.cpu_count(logical=False)  # 物理核心数
    if cpu_count is None:
        cpu_count = psutil.cpu_count()  # 逻辑核心数
    
    # workers数量：通常设置为CPU核心数的1/2到1/4
    workers = max(1, min(8, cpu_count // 2))
    return workers

# 估算最佳参数
batch_size = estimate_batch_size()
num_workers = estimate_workers()

print(f"Estimated optimal batch size: {batch_size}")
print(f"Estimated optimal num_workers: {num_workers}")

# 显示当前系统资源状态
print("\nSystem Resources:")
print(f"CPU Usage: {psutil.cpu_percent()}%")
mem = psutil.virtual_memory()
print(f"Memory: {mem.percent}% used, {mem.available/1024**3:.1f}GB available")
if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.memory_allocated()/1024**3:.1f}GB allocated")

# 配置 TensorBoard
try:
    from torch.utils.tensorboard import SummaryWriter
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorboard"])  # 安装 tensorboard
    from torch.utils.tensorboard import SummaryWriter

# 全局变量来存储 writer 和日志目录
writer = None
TB_LOG_DIR = None

def setup_tensorboard(phase_name: str):
    """为训练阶段设置 TensorBoard。"""
    global writer, TB_LOG_DIR
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    TB_LOG_DIR = Path(f"runs/tensorboard/{phase_name}/{timestamp}")
    TB_LOG_DIR.mkdir(parents=True, exist_ok=True)
    writer = SummaryWriter(log_dir=str(TB_LOG_DIR))
    print(f"TensorBoard for '{phase_name}' phase initialized. Logs will be saved to: {TB_LOG_DIR}")


def _safe_scalar_log(prefix: str, data: dict, step: int):
    if not isinstance(data, dict) or writer is None:
        return
    for k, v in data.items():
        if isinstance(v, (int, float)):
            writer.add_scalar(f"{prefix}/{k}", v, step)


def tb_on_fit_epoch_end(trainer):
    """每个 epoch 结束时记录 metrics、loss、学习率与显存占用。"""
    if writer is None: return
    epoch = getattr(trainer, "epoch", None)
    if epoch is None:
        return

    # 训练/验证指标
    metrics = getattr(trainer, "metrics", None) or {}
    _safe_scalar_log("metrics", metrics, epoch)

    # 训练损失（有的版本将损失汇总到 metrics 中）
    losses = getattr(trainer, "loss", None)
    if isinstance(losses, dict):
        _safe_scalar_log("loss", losses, epoch)

    # 学习率
    try:
        if getattr(trainer, "optimizer", None):
            lr = trainer.optimizer.param_groups[0].get("lr", None)
            if isinstance(lr, (int, float)):
                writer.add_scalar("opt/lr", lr, epoch)
    except Exception:
        pass

    # GPU 显存（如可用）
    try:
        if torch.cuda.is_available():
            mem = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
            writer.add_scalar("gpu/memory_allocated_mb", mem, epoch)
    except Exception:
        pass

    writer.flush()


def tb_on_train_end(trainer):
    """训练结束时的回调。"""
    global writer
    if writer:
        writer.close()
        print(f"TensorBoard logs saved to: {TB_LOG_DIR}")
        writer = None # 重置 writer
    
    # 训练结束后清理内存
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("TensorBoard callback functions defined and setup_tensorboard function is ready.")


Estimated optimal batch size: 32
Estimated optimal num_workers: 4

System Resources:
CPU Usage: 7.2%
Memory: 70.3% used, 4.5GB available
GPU Memory: 0.0GB allocated
TensorBoard callback functions defined and setup_tensorboard function is ready.


In [ ]:
# 加载模型并开始训练
from ultralytics import YOLO
import torch

# 1. 设置此阶段的 TensorBoard
setup_tensorboard("warmup")

# 2. 加载预训练模型
model = YOLO('yolo11n.pt')

# 3. 注册 TensorBoard 回调
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end) # 使用 on_train_end
except NameError:
    print("TensorBoard callbacks not found. Please run the previous cell that defines them.")

# 4. 动态设备选择
_device = 0 if torch.cuda.is_available() else 'cpu'

# 5. 开始训练
results = model.train(
    data='pest24.yaml',
    epochs=20,                # warmup周期
    lr0=1e-4,                # 初始学习率
    dropout=0.2,             # Dropout
    weight_decay=0.01,       # 权重衰减
    patience=3,              # 早停策略
    batch=int(batch_size/2),      # 降低 batch_size
    workers=int(num_workers/2),     # 降低 workers
    device=_device,          # 设备选择
    freeze=10,              # 冻结层数
    cache=True,            # 启用缓存
    overlap_mask=False,     # 减少mask计算开销
    profile=False,          # 关闭性能分析
    amp=True               # 自动混合精度训练
)

# 6. 保存warmup权重
model.save('yolo11n_warmup.pt')

print("\nWarmup complete. Launch TensorBoard with:\n  tensorboard --logdir runs/tensorboard\nThen open http://localhost:6006 in your browser.")


Ultralytics 8.3.208  Python-3.12.1 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=pest24.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=False, patience=3, perspective=0.0, plots=True, pose

In [5]:
# 阶段二：Finetune 训练
from ultralytics import YOLO
import torch

# 1. 设置此阶段的 TensorBoard
setup_tensorboard("finetune")

# 2. 加载 warmup 阶段训练好的模型
model = YOLO('ultralytics/runs/detect/train11/weights/best.pt')

# 3. 注册 TensorBoard 回调
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end) # 使用 on_train_end
except NameError:
    print("TensorBoard callbacks not found. Please run the cell that defines them.")

# 4. 动态设备选择
_device = 0 if torch.cuda.is_available() else 'cpu'

# 5. 开始 Finetune 训练
# 解冻所有层，使用更大的学习率
results = model.train(
    seed=42,
    data='pest24.yaml',
    epochs=100,               # 更多周期
    lr0=3e-2,                # 更大的学习率
    lrf=0.05,                 # 学习率衰减到 5%
    optimizer='AdamW',      # 使用 AdamW 优化器
    weight_decay=0.0005,       # 权重衰减
    dropout=0.2,             # Dropout
    cos_lr=False,            # 余弦退火学习率
    patience=15,             # 更多耐心
    batch=24,        # 使用估算的最佳 batch_size
    workers=2,     # 使用估算的最佳 workers
    device=_device,
    freeze=0,                # 解冻所有层
    cache=True,
    amp=True,
    overlap_mask=False,      # 减少mask计算开销
    save_period=10,           # 周期保存模型
    resume=False
)

# 6. 保存最终模型
model.save('yolo11n_final2.pt')

print("\nFinetune complete. Launch TensorBoard with:\n  tensorboard --logdir runs/tensorboard\nThen open http://localhost:6006 in your browser.")


TensorBoard for 'finetune' phase initialized. Logs will be saved to: runs\tensorboard\finetune\20251012_003228
New https://pypi.org/project/ultralytics/8.3.209 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.208  Python-3.12.1 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=pest24.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.03, lrf=0.05, mask_ratio=4, max_det=300, mixup=0.0, mode=tra